In [36]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [31]:
MINES = ['MINE1', 'MINE2', 'MINE3', 'MINE4']
YEARS = ['YEAR1', 'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5']
NEXT_YEAR = {'YEAR1': 'YEAR2', 
             'YEAR2': 'YEAR3',
             'YEAR3': 'YEAR4',
             'YEAR4': 'YEAR5'}

royalties = dict(zip(MINES, [5e6, 4e6, 4e6, 5e6]))
ore_limit = dict(zip(MINES, [2e6, 2.5e6, 1.3e6, 3e6]))
ore_quali = dict(zip(MINES, [1.0, 0.7, 1.5, 0.5]))
req_quali = dict(zip(YEARS, [0.9, 0.8, 1.2, 0.6, 1.0]))
year_disc = dict(zip(YEARS, [(1/(1+1/10.0))**(t-1) for t in [1,2,3,4,5]]))

blend_price   = 10
rev_disc_rate = 0.1

In [32]:
years = [1, 2, 3, 4, 5]
aa = {year: (1/(1+1/10.0)) ** (year-1) for year in years}
year_disc

{'YEAR1': 1.0,
 'YEAR2': 0.9090909090909091,
 'YEAR3': 0.8264462809917354,
 'YEAR4': 0.7513148009015777,
 'YEAR5': 0.6830134553650706}

In [33]:
model = gp.Model('Mining')

# add vars
extract = model.addVars(MINES, YEARS,
                        name='extract')
make = model.addVars(YEARS,
                     name='make')
used = model.addVars(MINES, YEARS,
                     name='used',
                     vtype=gp.GRB.BINARY)
active = model.addVars(MINES, YEARS,
                       name='active',
                       vtype=gp.GRB.BINARY)

# objective function
model.setObjective(gp.quicksum(blend_price*year_disc[t]*make[t] for t in YEARS)
                   - gp.quicksum(royalties[m]*year_disc[t]*active[m,t] for m in MINES for t in YEARS),
                   GRB.MAXIMIZE)

# add constraints
model.addConstrs((gp.quicksum(used[m,t] for m in MINES) <= 3 for t in YEARS),
                 name='mines_limit')
model.addConstrs((extract[m,t] - ore_limit[m]*used[m,t] <= 0 for m in MINES for t in YEARS),
                 name='extract_then_used')
model.addConstrs((used[m,t] - active[m,t] <= 0 for m in MINES for t in YEARS),
                 name='notactive_then_cantbeused')
model.addConstrs((active[m,NEXT_YEAR[t]] - active[m,t] <= 0 for m in MINES for t in YEARS[:4]),
                 name='notactive_then_notactiveanymore')
model.addConstrs((gp.quicksum(ore_quali[m]*extract[m,t] for m in MINES)
                              - req_quali[t]*make[t] == 0 for t in YEARS),
                 name='quality')
model.addConstrs((gp.quicksum(extract[m,t] for m in MINES) - make[t] == 0 for t in YEARS),
                 name='mass_conservation')
model.update()

In [34]:
model.write('Mining.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71 rows, 65 columns and 182 nonzeros
Model fingerprint: 0xf2903faa
Variable types: 25 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+06]
  Objective range  [7e+00, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 13 columns
Presolve time: 0.00s
Presolved: 58 rows, 52 columns, 135 nonzeros
Variable types: 16 continuous, 36 integer (36 binary)

Root relaxation: objective 1.577309e+08, 40 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.5773e+08    0    4   -0.00000 1.5773e+08      -     -    0s
H    0     0                    1.189074e+08 1.5773e+08  32.7%

In [68]:
print('Profit of ${:.2f}'.format(model.objVal))

make_solution = pd.DataFrame([], columns=['tons'], index=YEARS)
make_solution = make_solution.fillna(0)
for t in make.keys():
    make_solution['tons'][t] = make[t].x
print('===PRODUCING PLAN')
print(make_solution)

extract_solution = pd.DataFrame([], columns=MINES, index=YEARS)
extract_solution = extract_solution.fillna(0)
for m, t in extract.keys():
    extract_solution[m][t] = extract[m,t].x
print('===EXTRACTION PLAN')
print(extract_solution)

Profit of $146861974.36
===PRODUCING PLAN
          tons
YEAR1  5750000
YEAR2  5999999
YEAR3  3250000
YEAR4  5625000
YEAR5  5466666
===EXTRACTION PLAN
         MINE1    MINE2    MINE3    MINE4
YEAR1  2000000        0  1300000  2450000
YEAR2        0  2500000  1300000  2199999
YEAR3  1950000        0  1300000        0
YEAR4   124999  2500000        0  3000000
YEAR5  2000000  2166666  1300000        0
